In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import load

In [26]:
xg_enn = load('modelos/xg_enn_100.joblib')
xg_tomek = load('modelos/xg_tomek_100.joblib')
xg_smt = load('modelos/xg_smt_100.joblib')
rf_enn = load('modelos/rf_enn_100.joblib')
rf_tomek = load('modelos/rf_tomek_100.joblib')
rf_smt = load('modelos/rf_smt_100.joblib')
muestra = pd.read_csv('archivos/estrellas aleatorias por colores/muestra_con_simbad.csv')
muestra_filtrada =  muestra[ muestra['otype'] == '   *']

C:\Users\diego\AppData\Local\Temp\ipykernel_29120\832590129.py:7: DtypeWarning: Columns (189) have mixed types. Specify dtype option on import or set low_memory=False.
  muestra = pd.read_csv('archivos/estrellas aleatorias por colores/muestra_con_simbad.csv')


In [27]:
print(list(muestra_filtrada.columns))
muestra_filtrada.shape

['Unnamed: 0', 'source_id', 'solution_id', 'designation', 'source_id.1', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pm', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'nu_eff_used_in_astrometry', 'pseudocolour', 'pseudocolour_error', 'ra_pseudocolour_corr', 'dec_pseudocolour_corr', 'parallax_pseudocolour_corr', 'pmra_pseudocolour_corr', 'pmdec_pseudocolour_corr', 'astrometric_matched_transits', 'visibility_periods_used', 'astrometric_sigma5d_max', 'matched_transi

(149955, 212)

In [28]:
muestra_filtrada['GAIA_G-GAIA_BP'] = muestra_filtrada['GAIA_G'] - muestra_filtrada['GAIA_BP']
muestra_filtrada['GAIA_G-GAIA_RP'] = muestra_filtrada['GAIA_G'] - muestra_filtrada['GAIA_RP']
muestra_filtrada['GAIA_BP-GAIA_RP'] = muestra_filtrada['GAIA_BP'] - muestra_filtrada['GAIA_RP']
muestra_filtrada['J-H'] = muestra_filtrada['J'] - muestra_filtrada['H']
muestra_filtrada['J-K'] = muestra_filtrada['J'] - muestra_filtrada['K']
muestra_filtrada['H-K'] = muestra_filtrada['H'] - muestra_filtrada['K']

C:\Users\diego\AppData\Local\Temp\ipykernel_29120\3700395615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muestra_filtrada['GAIA_G-GAIA_BP'] = muestra_filtrada['GAIA_G'] - muestra_filtrada['GAIA_BP']
C:\Users\diego\AppData\Local\Temp\ipykernel_29120\3700395615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muestra_filtrada['GAIA_G-GAIA_RP'] = muestra_filtrada['GAIA_G'] - muestra_filtrada['GAIA_RP']
C:\Users\diego\AppData\Local\Temp\ipykernel_29120\3700395615.py:3: SettingWithCopyWarning: 
A valu

In [29]:
columnas = ['parallax', 'GAIA_G-GAIA_BP', 'GAIA_G-GAIA_RP', 'GAIA_BP-GAIA_RP', 'J-H', 'J-K', 'H-K']
muestra_filtrada_colores = muestra_filtrada[columnas]

In [37]:
prob_enn = xg_enn.predict_proba(muestra_filtrada_colores)[:, 1]
prob_tomek = xg_tomek.predict_proba(muestra_filtrada_colores)[:, 1]
prob_smt = xg_smt.predict_proba(muestra_filtrada_colores)[:, 1]
rf_prob_enn = rf_enn.predict_proba(muestra_filtrada_colores)[:, 1]
rf_prob_tomek = rf_tomek.predict_proba(muestra_filtrada_colores)[:, 1]
rf_prob_smt = rf_smt.predict_proba(muestra_filtrada_colores)[:, 1]

ponderacion = (prob_enn + prob_tomek + prob_smt + rf_prob_enn + rf_prob_tomek + rf_prob_smt) / 6

# Filtrar las filas con ponderación mayor o igual a 0.5
filtrado = ponderacion >= 0.85

# Crear un nuevo DataFrame con 'source_id' y las ponderaciones
resultado = pd.DataFrame({
    'source_id': muestra_filtrada['source_id'],  # Tomar el 'source_id' de muestra_filtrada original
    'ponderacion': ponderacion
})

# Filtrar los resultados donde la ponderación sea >= 0.5
resultado_filtrado = resultado[filtrado]
resultado_ordenado = resultado_filtrado.sort_values(by='ponderacion', ascending=False)

print(resultado_ordenado.head())
resultado_ordenado.to_csv('archivos/datos finales/predicciones_filtradas.csv', index=False)
resultado_ordenado.shape

                  source_id  ponderacion
35108   3352737959020047360     0.997329
189254  5350349479166230144     0.996353
196299  5258613307042931968     0.995904
45418   4050252261569856896     0.993503
90502   3107767215029913088     0.993182


(270, 2)